In [32]:
import json
import torch
from PIL import Image
from tqdm.auto import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
from torch.utils.data import DataLoader
from transformers import AutoModelForObjectDetection

# 이미 학습된 모델을 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForObjectDetection.from_pretrained(
    "C:/Users/fun67/Desktop/lastpj/ckp/ckpt_rtdetr_r50vd_n98epoch"
).eval().to(device)

# COCO 데이터셋 로드
with open("./train.json", "r", encoding="utf-8") as f:
    data_list = json.load(f)

# 전처리 함수 정의
def custom_preprocess(image: Image.Image, target_size=(640, 640)):
    image = image.convert("RGB")  # 이미지가 RGB로 변환되도록 보장
    image = image.resize(target_size)  # 크기 조정
    image = np.array(image).astype(np.float32)  # numpy 배열로 변환 및 float32로 변환
    image = image / 255.0  # 정규화
    image = np.transpose(image, (2, 0, 1))  # 채널을 첫 번째로 (C, H, W) 형식으로 변경
    return torch.tensor(image)  # torch.Tensor로 변환

# COCO 형식으로 변환하는 함수
def convert_to_coco_format(data_list):
    coco_annotations = {
        "images": [],
        "annotations": [],
        "categories": [
            {"id": 0, "name": "crack"},
            {"id": 1, "name": "pothole"},
        ]
    }
    
    annotation_id = 1
    for idx, data in enumerate(data_list):
        coco_annotations["images"].append({
            "file_name": data["image"],
            "height": data["height"],
            "width": data["width"],
            "id": int(idx)  # int64 -> int 변환
        })
        
        for obj_id, bbox, area, category in zip(data["objects"]["id"], data["objects"]["bbox"], data["objects"]["area"], data["objects"]["category"]):
            coco_annotations["annotations"].append({
                "image_id": int(idx),  # int64 -> int 변환
                "category_id": int(category),  # int64 -> int 변환
                "bbox": [float(i) for i in bbox],  # bbox 값은 float로 변환
                "area": float(area),  # area는 float로 변환
                "iscrowd": 0,  # 일반적으로 0
                "id": int(annotation_id)  # int64 -> int 변환
            })
            annotation_id += 1

    return coco_annotations

# 일부 이미지에 대한 mAP 추론을 위한 함수
def evaluate_map(model, data_list, num_samples=30):
    model.eval()

    # 일부 샘플만 사용
    selected_samples = data_list[:num_samples]

    # COCO 형식으로 변환
    coco_annotations = convert_to_coco_format(selected_samples)

    # 모델의 예측 결과 저장
    all_predictions = []

    # COCO 데이터셋 로드 (COCO API)
    coco = COCO()
    coco.dataset = coco_annotations
    coco.createIndex()

    # 예측 및 실제 라벨 비교
    for idx, sample in enumerate(selected_samples):
        image_path = sample["image"]
        image = Image.open(image_path)
        processed_image = custom_preprocess(image).unsqueeze(0).to(device)  # 배치 차원 추가

        with torch.no_grad():
            outputs = model(pixel_values=processed_image)


        # 예측된 경계 상자 및 점수 추출 (출력 형식에 맞게 수정)
        pred_boxes = outputs["pred_boxes"][0].cpu().numpy()  # 예측된 경계 상자
        pred_scores = outputs["logits"].softmax(-1)[0].cpu().numpy()  # 클래스 점수

        # 가장 높은 점수를 가진 클래스 선택
        for box, scores in zip(pred_boxes, pred_scores):
            category_id = np.argmax(scores[:-1])  # 배경 클래스 제외
            score = np.max(scores[:-1])  # 배경 클래스 제외

            if score > 0:  # 일정 점수 이상일 때만 추가
                all_predictions.append({
                    "image_id": int(idx),  # int64 -> int 변환
                    "category_id": int(category_id),  # int64 -> int 변환
                    "bbox": [float(i) for i in box],  # bbox 값은 float로 변환
                    "score": float(score)  # score는 float로 변환
                })

    # COCO 포맷으로 예측 저장
    with open("predictions.json", "w") as f:
        json.dump(all_predictions, f)

    # COCO API로 mAP 평가
    coco_pred = coco.loadRes("predictions.json")
    coco_eval = COCOeval(coco, coco_pred, iouType="bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()



# 5개의 이미지에 대해 mAP를 평가
evaluate_map(model, data_list, num_samples=30)


creating index...
index created!
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.5

In [33]:
import os
import random
import torch
from PIL import Image, ImageDraw
from transformers import AutoModelForObjectDetection
import numpy as np

# 경로 설정
image_folder = "C:/Users/fun67/Desktop/lastpj/data/train/images"
label_folder = "C:/Users/fun67/Desktop/lastpj/data/train/labels"

# 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForObjectDetection.from_pretrained(
    "C:/Users/fun67/Desktop/lastpj/ckp/ckpt_rtdetr_r50vd_n98epoch"
).eval().to(device)

# COCO 카테고리
id2label = {0: "crack", 1: "pothole"}  # 각 category_id에 해당하는 라벨 이름

# 전처리 함수 정의
def custom_preprocess(image: Image.Image, target_size=(640, 640)):
    image = image.convert("RGB")
    image = image.resize(target_size)
    image = np.array(image).astype(np.float32) / 255.0
    image = np.transpose(image, (2, 0, 1))  # 채널을 첫 번째로 (C, H, W) 형식으로 변경
    return torch.tensor(image).unsqueeze(0)  # 배치 차원 추가

# YOLO 라벨 파일을 읽어서 경계 상자를 그리기
def draw_yolo_labels(image, label_file, id2label):
    image_width, image_height = image.size
    with open(label_file, "r") as f:
        labels = f.readlines()
    
    draw = ImageDraw.Draw(image)

    for label in labels:
        class_id, x_center, y_center, box_width, box_height = map(float, label.split())

        # YOLO 형식의 상대 좌표를 절대 좌표로 변환
        x_center *= image_width
        y_center *= image_height
        box_width *= image_width
        box_height *= image_height

        # 좌상단 (x_min, y_min) 및 우하단 (x_max, y_max) 계산
        x_min = x_center - (box_width / 2)
        y_min = y_center - (box_height / 2)
        x_max = x_center + (box_width / 2)
        y_max = y_center + (box_height / 2)

        # 빨간색으로 정답 상자 그리기
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
        draw.text((x_min, y_min), id2label[int(class_id)], fill="red")  # 정답 상자는 빨간색

# 예측 상자 그리기
def draw_predictions(image, pred_boxes, pred_scores, id2label, threshold=0.3):
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size  # 이미지의 실제 크기 가져오기
    for box, scores in zip(pred_boxes, pred_scores):
        max_score = np.max(scores)  # 클래스별 점수 중 최대값
        if max_score > threshold:
            category_id = np.argmax(scores[:-1])  # 배경 클래스 제외
            
            # 모델이 반환하는 좌표를 절대 좌표로 변환 (상대 좌표일 경우)
            # 예: 모델 출력이 상대 좌표일 경우, 이미지 크기에 맞춰 변환 필요
            x_min, y_min, box_width, box_height = box
            x_min *= image_width
            y_min *= image_height
            box_width *= image_width
            box_height *= image_height

            x_max = x_min + box_width
            y_max = y_min + box_height

            draw.rectangle((x_min, y_min, x_max, y_max), outline="blue", width=2)
            draw.text((x_min, y_min), id2label[category_id], fill="blue")  # 예측 상자는 파란색

# 이미지와 라벨 파일 랜덤으로 5개 선택
image_files = os.listdir(image_folder)
random_files = random.sample(image_files, 5)  # 이미지 파일에서 랜덤으로 5개 선택

for image_file_name in random_files:
    image_file = os.path.join(image_folder, image_file_name)
    label_file_name = image_file_name.replace(".jpg", ".txt")  # 라벨 파일은 같은 이름의 .txt 파일
    label_file = os.path.join(label_folder, label_file_name)
    
    # 이미지 로드
    image = Image.open(image_file)
    
    # 이미지 전처리
    processed_image = custom_preprocess(image).to(device)

    # 모델을 사용한 예측
    with torch.no_grad():
        outputs = model(pixel_values=processed_image)

    # 예측된 경계 상자 및 점수 추출
    pred_boxes = outputs["pred_boxes"][0].cpu().numpy()  # 예측된 경계 상자
    pred_scores = outputs["logits"].softmax(-1)[0].cpu().numpy()  # 클래스 점수

    # 정답 상자 그리기 (빨간색, YOLO 형식 라벨)
    draw_yolo_labels(image, label_file, id2label)

    # 예측 상자 그리기 (파란색)
    draw_predictions(image, pred_boxes, pred_scores, id2label, threshold=0.3)

    # 최종 결과 출력
    image.show()


#h1 단일 모델 NMS 

In [27]:
import os
import random
import torch
from PIL import Image, ImageDraw
from transformers import AutoModelForObjectDetection
import numpy as np
import torchvision

# 경로 설정
image_folder = "C:/Users/fun67/Desktop/lastpj/data/train/images"
label_folder = "C:/Users/fun67/Desktop/lastpj/data/train/labels"

# 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForObjectDetection.from_pretrained(
    "ckp/ckpt_rtdetr_r50vd_n98epoch"
).eval().to(device)

# COCO 카테고리 id2label
id2label = {0: "crack", 1: "pothole"}  # 각 category_id에 해당하는 라벨 이름

# 전처리 함수 정의
def custom_preprocess(image: Image.Image, target_size=(640, 640)):
    image = image.convert("RGB")
    image = image.resize(target_size)
    image = np.array(image).astype(np.float32) / 255.0
    image = np.transpose(image, (2, 0, 1))  # 채널을 첫 번째로 (C, H, W) 형식으로 변경
    return torch.tensor(image).unsqueeze(0)  # 배치 차원 추가

# YOLO 라벨 파일을 읽어서 경계 상자를 그리기
def draw_yolo_labels(image, label_file, id2label):
    image_width, image_height = image.size
    with open(label_file, "r") as f:
        labels = f.readlines()

    draw = ImageDraw.Draw(image)

    for label in labels:
        class_id, x_center, y_center, box_width, box_height = map(float, label.split())

        # YOLO 형식의 상대 좌표를 절대 좌표로 변환
        x_center *= image_width
        y_center *= image_height
        box_width *= image_width
        box_height *= image_height

        # 좌상단 (x_min, y_min) 및 우하단 (x_max, y_max) 계산
        x_min = x_center - (box_width / 2)
        y_min = y_center - (box_height / 2)
        x_max = x_center + (box_width / 2)
        y_max = y_center + (box_height / 2)

        # 빨간색으로 정답 상자 그리기
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
        draw.text((x_min, y_min), id2label[int(class_id)], fill="red")  # 정답 상자는 빨간색

# NMS 적용 함수
def apply_nms(pred_boxes, pred_scores, iou_threshold=0.5):
    # 최대 점수와 상응하는 클래스 id를 가져옴
    scores = np.max(pred_scores[:, :-1], axis=1)  # 배경 클래스 제외
    boxes = torch.tensor(pred_boxes)  # 예측된 박스 (x_min, y_min, w, h)
    
    # (x_min, y_min, x_max, y_max) 형식으로 변환
    pred_boxes_xyxy = torch.zeros_like(boxes)
    pred_boxes_xyxy[:, 0] = boxes[:, 0]  # x_min
    pred_boxes_xyxy[:, 1] = boxes[:, 1]  # y_min
    pred_boxes_xyxy[:, 2] = boxes[:, 0] + boxes[:, 2]  # x_max = x_min + width
    pred_boxes_xyxy[:, 3] = boxes[:, 1] + boxes[:, 3]  # y_max = y_min + height

    print("Boxes before NMS:", pred_boxes_xyxy)
    print("Scores before NMS:", scores)

    # NMS 적용 (iou_threshold는 겹치는 비율의 임계값)
    keep_indices = torchvision.ops.nms(pred_boxes_xyxy, torch.tensor(scores), iou_threshold)
    
    print("Kept indices after NMS:", keep_indices)

    # NMS 결과 반환 (필터링된 상자 및 점수)
    return pred_boxes[keep_indices], pred_scores[keep_indices]

# 예측 상자 그리기 (NMS 적용 후)
# 예측 상자 그리기 (NMS 적용 후)
def draw_predictions_with_nms(image, pred_boxes, pred_scores, id2label, threshold=0.1, iou_threshold=0.1):
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size

    # NMS 적용
    filtered_boxes, filtered_scores = apply_nms(pred_boxes, pred_scores, iou_threshold)

    for box, scores in zip(filtered_boxes, filtered_scores):
        max_score = np.max(scores)
        if max_score > threshold:
            category_id = np.argmax(scores[:-1])  # 배경 클래스 제외

            # 예측 상자 변환 (YOLO 형식처럼)
            x_min, y_min, box_width, box_height = box

            # 좌표 변환 (중앙 좌표와 크기로부터 좌상단과 우하단 좌표 계산)
            x_center = x_min
            y_center = y_min
            x_min = (x_center - box_width / 2) * image_width
            y_min = (y_center - box_height / 2) * image_height
            x_max = (x_center + box_width / 2) * image_width
            y_max = (y_center + box_height / 2) * image_height

            # 파란색으로 예측 상자 그리기
            draw.rectangle((x_min, y_min, x_max, y_max), outline="blue", width=2)
            draw.text((x_min, y_min), id2label[category_id], fill="blue")


# 이미지와 라벨 파일 랜덤으로 5개 선택
image_files = os.listdir(image_folder)
random_files = random.sample(image_files, 1)  # 이미지 파일에서 랜덤으로 5개 선택

for image_file_name in random_files:
    image_file = os.path.join(image_folder, image_file_name)
    label_file_name = image_file_name.replace(".jpg", ".txt")  # 라벨 파일은 같은 이름의 .txt 파일
    label_file = os.path.join(label_folder, label_file_name)
    
    # 원본 파일 경로 출력
    print(f"Image file path: {image_file}")
    print(f"Label file path: {label_file}")

    # 이미지 로드
    image = Image.open(image_file)

    # 이미지 전처리
    processed_image = custom_preprocess(image).to(device)

    # 모델을 사용한 예측
    with torch.no_grad():
        outputs = model(pixel_values=processed_image)

    # 예측된 경계 상자 및 점수 추출
    pred_boxes = outputs["pred_boxes"][0].cpu().numpy()  # 예측된 경계 상자
    pred_scores = outputs["logits"].softmax(-1)[0].cpu().numpy()  # 클래스 점수

    # 정답 상자 그리기 (빨간색, YOLO 형식 라벨)
    draw_yolo_labels(image, label_file, id2label)

    # 예측 상자 그리기 (NMS 적용하여 파란색 상자만 표시)
    draw_predictions_with_nms(image, pred_boxes, pred_scores, id2label, threshold=0.5, iou_threshold=0.97)

    # 최종 결과 출력
    image.show()

Image file path: C:/Users/fun67/Desktop/lastpj/data/train/images\22232a917e1a165ecb378fb3.jpg
Label file path: C:/Users/fun67/Desktop/lastpj/data/train/labels\22232a917e1a165ecb378fb3.txt
Boxes before NMS: tensor([[0.3223, 0.3066, 0.5225, 0.8742],
        [0.0698, 0.2731, 0.1546, 0.9186],
        [0.6759, 0.1309, 0.7004, 0.1649],
        ...,
        [0.1967, 0.0284, 0.2536, 0.0850],
        [0.9593, 0.4394, 0.9893, 0.4758],
        [0.1875, 0.3882, 0.2420, 0.4205]])
Scores before NMS: [0.9566555  0.8337832  0.9829768  0.92811173 0.8855311  0.79913026
 0.9403639  0.9131748  0.9711592  0.9492621  0.9633989  0.8704058
 0.9156393  0.99446714 0.95218843 0.67683095 0.9375367  0.8925924
 0.88898647 0.9004191  0.8502052  0.89624494 0.90779346 0.94502115
 0.9215011  0.9009004  0.8059885  0.85028344 0.89495975 0.83309114
 0.7559803  0.92277133 0.84793174 0.7726171  0.9500346  0.95296675
 0.9056214  0.8789002  0.910765   0.9253096  0.8743888  0.9398338
 0.9543219  0.89949733 0.73579067 0.7472597

NMS한거 안한거 단일 파일 이미지 2개

In [29]:
import os
import random
import torch
from PIL import Image, ImageDraw
from transformers import AutoModelForObjectDetection
import numpy as np
import torchvision

# 경로 설정
image_folder = "C:/Users/fun67/Desktop/lastpj/data/train/images"
label_folder = "C:/Users/fun67/Desktop/lastpj/data/train/labels"

# 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForObjectDetection.from_pretrained(
    "ckp/ckpt_rtdetr_r50vd_n98epoch"
).eval().to(device)

# COCO 카테고리 id2label
id2label = {0: "crack", 1: "pothole"}

# 전처리 함수 정의
def custom_preprocess(image: Image.Image, target_size=(640, 640)):
    image = image.convert("RGB")
    image = image.resize(target_size)
    image = np.array(image).astype(np.float32) / 255.0
    image = np.transpose(image, (2, 0, 1))  # 채널을 첫 번째로 (C, H, W) 형식으로 변경
    return torch.tensor(image).unsqueeze(0)  # 배치 차원 추가

# YOLO 라벨 파일을 읽어서 경계 상자를 그리기
def draw_yolo_labels(image, label_file, id2label):
    image_width, image_height = image.size
    with open(label_file, "r") as f:
        labels = f.readlines()

    draw = ImageDraw.Draw(image)

    for label in labels:
        class_id, x_center, y_center, box_width, box_height = map(float, label.split())

        # YOLO 형식의 상대 좌표를 절대 좌표로 변환
        x_center *= image_width
        y_center *= image_height
        box_width *= image_width
        box_height *= image_height

        # 좌상단 (x_min, y_min) 및 우하단 (x_max, y_max) 계산
        x_min = x_center - (box_width / 2)
        y_min = y_center - (box_height / 2)
        x_max = x_center + (box_width / 2)
        y_max = y_center + (box_height / 2)

        # 빨간색으로 정답 상자 그리기
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
        draw.text((x_min, y_min), id2label[int(class_id)], fill="red")

# NMS 적용 함수
def apply_nms(pred_boxes, pred_scores, iou_threshold=0.5):
    # 최대 점수와 상응하는 클래스 id를 가져옴
    scores = np.max(pred_scores[:, :-1], axis=1)  # 배경 클래스 제외
    boxes = torch.tensor(pred_boxes)  # 예측된 박스 (x_min, y_min, w, h)

    # (x_min, y_min, x_max, y_max) 형식으로 변환
    pred_boxes_xyxy = torch.zeros_like(boxes)
    pred_boxes_xyxy[:, 0] = boxes[:, 0]  # x_min
    pred_boxes_xyxy[:, 1] = boxes[:, 1]  # y_min
    pred_boxes_xyxy[:, 2] = boxes[:, 0] + boxes[:, 2]  # x_max = x_min + width
    pred_boxes_xyxy[:, 3] = boxes[:, 1] + boxes[:, 3]  # y_max = y_min + height

    # NMS 적용 (iou_threshold는 겹치는 비율의 임계값)
    keep_indices = torchvision.ops.nms(pred_boxes_xyxy, torch.tensor(scores), iou_threshold)

    # NMS 결과 반환 (필터링된 상자 및 점수)
    return pred_boxes[keep_indices], pred_scores[keep_indices]

# 예측 상자 그리기 (NMS 적용 후)
def draw_predictions_with_nms(image, pred_boxes, pred_scores, id2label, threshold=0.1, iou_threshold=0.9):
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size

    # NMS 적용
    filtered_boxes, filtered_scores = apply_nms(pred_boxes, pred_scores, iou_threshold)

    for box, scores in zip(filtered_boxes, filtered_scores):
        max_score = np.max(scores)
        if max_score > threshold:
            category_id = np.argmax(scores[:-1])  # 배경 클래스 제외

            # 예측 상자 변환 (YOLO 형식처럼)
            x_min, y_min, box_width, box_height = box

            # 좌표 변환 (중앙 좌표와 크기로부터 좌상단과 우하단 좌표 계산)
            x_center = x_min
            y_center = y_min
            x_min = (x_center - box_width / 2) * image_width
            y_min = (y_center - box_height / 2) * image_height
            x_max = (x_center + box_width / 2) * image_width
            y_max = (y_center + box_height / 2) * image_height

            # 파란색으로 예측 상자 그리기
            draw.rectangle((x_min, y_min, x_max, y_max), outline="blue", width=2)
            draw.text((x_min, y_min), id2label[category_id], fill="blue")

# 예측 상자 그리기 (NMS 미적용)
def draw_predictions_without_nms(image, pred_boxes, pred_scores, id2label, threshold=0.9):
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size

    for box, scores in zip(pred_boxes, pred_scores):
        max_score = np.max(scores)
        if max_score > threshold:
            category_id = np.argmax(scores[:-1])  # 배경 클래스 제외

            # 예측 상자 변환 (YOLO 형식처럼)
            x_min, y_min, box_width, box_height = box

            # 좌표 변환 (중앙 좌표와 크기로부터 좌상단과 우하단 좌표 계산)
            x_center = x_min
            y_center = y_min
            x_min = (x_center - box_width / 2) * image_width
            y_min = (y_center - box_height / 2) * image_height
            x_max = (x_center + box_width / 2) * image_width
            y_max = (y_center + box_height / 2) * image_height

            # 녹색으로 예측 상자 그리기 (NMS 없이)
            draw.rectangle((x_min, y_min, x_max, y_max), outline="green", width=2)
            draw.text((x_min, y_min), id2label[category_id], fill="green")

# 이미지와 라벨 파일 랜덤으로 5개 선택
image_files = os.listdir(image_folder)
random_files = random.sample(image_files, 1)  # 이미지 파일에서 랜덤으로 5개 선택

for image_file_name in random_files:
    image_file = os.path.join(image_folder, image_file_name)
    label_file_name = image_file_name.replace(".jpg", ".txt")  # 라벨 파일은 같은 이름의 .txt 파일
    label_file = os.path.join(label_folder, label_file_name)

    # 원본 파일 경로 출력
    print(f"Image file path: {image_file}")
    print(f"Label file path: {label_file}")

    # 이미지 로드
    image = Image.open(image_file)
    

    # 이미지 전처리
    processed_image = custom_preprocess(image).to(device)

    # 모델을 사용한 예측
    with torch.no_grad():
        outputs = model(pixel_values=processed_image)

    # 예측된 경계 상자 및 점수 추출
    pred_boxes = outputs["pred_boxes"][0].cpu().numpy()  # 예측된 경계 상자
    pred_scores = outputs["logits"].softmax(-1)[0].cpu().numpy()  # 클래스 점수

    # 정답 상자 그리기 (빨간색, YOLO 형식 라벨)
    draw_yolo_labels(image, label_file, id2label)

    # 예측 상자 그리기 (NMS 미적용, 녹색 상자)
    image_without_nms = image.copy()
    draw_predictions_without_nms(image_without_nms, pred_boxes, pred_scores, id2label, threshold=0.95)

    # 예측 상자 그리기 (NMS 적용하여 파란색 상자만 표시)
    image_with_nms = image.copy()
    draw_predictions_with_nms(image_with_nms, pred_boxes, pred_scores, id2label, threshold=0.5, iou_threshold=0.97)

    # NMS 없이 이미지 출력
    print("Displaying image without NMS:")
    image_without_nms.show()

    # NMS 적용된 이미지 출력
    print("Displaying image with NMS:")
    image_with_nms.show()


Image file path: C:/Users/fun67/Desktop/lastpj/data/train/images\ef9b949862519c483bf850c1.jpg
Label file path: C:/Users/fun67/Desktop/lastpj/data/train/labels\ef9b949862519c483bf850c1.txt
Displaying image without NMS:
Displaying image with NMS:


In [14]:
import os
import random
import torch
from PIL import Image, ImageDraw
from transformers import AutoModelForObjectDetection
import numpy as np

# 경로 설정
image_folder = "C:/Users/fun67/Desktop/lastpj/data/train/images"
label_folder = "C:/Users/fun67/Desktop/lastpj/data/train/labels"

# 두 개의 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 기존 모델
model_old = AutoModelForObjectDetection.from_pretrained(
    r"C:\Users\fun67\Desktop\lastpj\ckp\ckpt_rtdetr_r50vd_n98epoch"
).eval().to(device)

# 새로운 모델
model_new = AutoModelForObjectDetection.from_pretrained(
    r"C:\Users\fun67\Desktop\lastpj\ckp\ckpt_rtdetr_r50vd_1epoch"
).eval().to(device)

# COCO 카테고리
id2label = {0: "crack", 1: "pothole"}  # 각 category_id에 해당하는 라벨 이름

# 전처리 함수 정의
def custom_preprocess(image: Image.Image, target_size=(640, 640)):
    image = image.convert("RGB")
    image = image.resize(target_size)
    image = np.array(image).astype(np.float32) / 255.0
    image = np.transpose(image, (2, 0, 1))  # 채널을 첫 번째로 (C, H, W) 형식으로 변경
    return torch.tensor(image).unsqueeze(0)  # 배치 차원 추가

# YOLO 라벨 파일을 읽어서 경계 상자를 그리기
def draw_yolo_labels(image, label_file, id2label):
    image_width, image_height = image.size
    with open(label_file, "r") as f:
        labels = f.readlines()
    
    draw = ImageDraw.Draw(image)

    for label in labels:
        class_id, x_center, y_center, box_width, box_height = map(float, label.split())

        # YOLO 형식의 상대 좌표를 절대 좌표로 변환
        x_center *= image_width
        y_center *= image_height
        box_width *= image_width
        box_height *= image_height

        # 좌상단 (x_min, y_min) 및 우하단 (x_max, y_max) 계산
        x_min = x_center - (box_width / 2)
        y_min = y_center - (box_height / 2)
        x_max = x_center + (box_width / 2)
        y_max = y_center + (box_height / 2)

        # 빨간색으로 정답 상자 그리기
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
        draw.text((x_min, y_min), id2label[int(class_id)], fill="red")  # 정답 상자는 빨간색

# 모델 예측 상자 그리기 (각 모델마다 다른 색상)
# 모델 예측 상자 그리기 (각 모델마다 다른 색상)
def draw_predictions(image, pred_boxes, pred_scores, id2label, color="blue", threshold=0.96):
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size  # 이미지의 실제 크기 가져오기

    for box, scores in zip(pred_boxes, pred_scores):
        max_score = np.max(scores)  # 클래스별 점수 중 최대값
        if max_score > threshold:
            category_id = np.argmax(scores[:-1])  # 배경 클래스 제외
            
            # 모델이 반환하는 좌표가 (x_center, y_center, width, height) 형식일 경우
            x_center, y_center, box_width, box_height = box
            
            # 좌상단 (x_min, y_min) 및 우하단 (x_max, y_max) 계산
            x_center *= image_width
            y_center *= image_height
            box_width *= image_width
            box_height *= image_height

            x_min = x_center - (box_width / 2)
            y_min = y_center - (box_height / 2)
            x_max = x_center + (box_width / 2)
            y_max = y_center + (box_height / 2)

            draw.rectangle([x_min, y_min, x_max, y_max], outline=color, width=2)
            draw.text((x_min, y_min), id2label[category_id], fill=color)

# 이미지와 라벨 파일 랜덤으로 5개 선택
image_files = os.listdir(image_folder)
random_files = random.sample(image_files, 5)  # 이미지 파일에서 랜덤으로 5개 선택

for image_file_name in random_files:
    image_file = os.path.join(image_folder, image_file_name)
    label_file_name = image_file_name.replace(".jpg", ".txt")  # 라벨 파일은 같은 이름의 .txt 파일
    label_file = os.path.join(label_folder, label_file_name)

    # 이미지 로드
    image = Image.open(image_file)

    # 이미지 전처리
    processed_image = custom_preprocess(image).to(device)

    # 기존 모델을 사용한 예측
    with torch.no_grad():
        outputs_old = model_old(pixel_values=processed_image)

    # 새로운 모델을 사용한 예측
    with torch.no_grad():
        outputs_new = model_new(pixel_values=processed_image)

    # 기존 모델의 예측 경계 상자 및 점수 추출
    pred_boxes_old = outputs_old["pred_boxes"][0].cpu().numpy()  # 예측된 경계 상자
    pred_scores_old = outputs_old["logits"].softmax(-1)[0].cpu().numpy()  # 클래스 점수

    # 새로운 모델의 예측 경계 상자 및 점수 추출
    pred_boxes_new = outputs_new["pred_boxes"][0].cpu().numpy()  # 예측된 경계 상자
    pred_scores_new = outputs_new["logits"].softmax(-1)[0].cpu().numpy()  # 클래스 점수

    # 정답 상자 그리기 (빨간색, YOLO 형식 라벨)
    draw_yolo_labels(image, label_file, id2label)
    Vth = 0.92
    # 기존 모델의 예측 상자 그리기 (파란색)
    draw_predictions(image, pred_boxes_old, pred_scores_old, id2label, color="blue", threshold=Vth)

    # 새로운 모델의 예측 상자 그리기 (초록색)
    draw_predictions(image, pred_boxes_new, pred_scores_new, id2label, color="green", threshold=Vth)

    # 최종 결과 출력
    image.show()



#다중

In [21]:
import os
import random
import torch
from PIL import Image, ImageDraw
from transformers import AutoModelForObjectDetection
import numpy as np

iouval = 0
# 경로 설정
image_folder = "C:/Users/fun67/Desktop/lastpj/data/train/images"
label_folder = "C:/Users/fun67/Desktop/lastpj/data/train/labels"

# 두 개의 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 기존 모델
model_old = AutoModelForObjectDetection.from_pretrained(
    "ckp/ckpt_rtdetr_r50vd_n98epoch"
).eval().to(device)

# 새로운 모델
model_new = AutoModelForObjectDetection.from_pretrained(
    "ckp/ckpt_rtdetr_r50vd_n71epoch"
).eval().to(device)

# COCO 카테고리
id2label = {0: "crack", 1: "pothole"}  # 각 category_id에 해당하는 라벨 이름

# 전처리 함수 정의
def custom_preprocess(image: Image.Image, target_size=(640, 640)):
    image = image.convert("RGB")
    image = image.resize(target_size)
    image = np.array(image).astype(np.float32) / 255.0
    image = np.transpose(image, (2, 0, 1))  # 채널을 첫 번째로 (C, H, W) 형식으로 변경
    return torch.tensor(image).unsqueeze(0)  # 배치 차원 추가

# YOLO 라벨 파일을 읽어서 경계 상자를 그리기
def draw_yolo_labels(image, label_file, id2label):
    image_width, image_height = image.size
    with open(label_file, "r") as f:
        labels = f.readlines()
    
    draw = ImageDraw.Draw(image)

    for label in labels:
        class_id, x_center, y_center, box_width, box_height = map(float, label.split())

        # YOLO 형식의 상대 좌표를 절대 좌표로 변환
        x_center *= image_width
        y_center *= image_height
        box_width *= image_width
        box_height *= image_height

        # 좌상단 (x_min, y_min) 및 우하단 (x_max, y_max) 계산
        x_min = x_center - (box_width / 2)
        y_min = y_center - (box_height / 2)
        x_max = x_center + (box_width / 2)
        y_max = y_center + (box_height / 2)

        # 빨간색으로 정답 상자 그리기
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
        draw.text((x_min, y_min), id2label[int(class_id)], fill="red")  # 정답 상자는 빨간색

# 모델 예측 상자 그리기 (각 모델마다 다른 색상)
def draw_predictions(image, pred_boxes, pred_scores, id2label, color="blue", threshold=0.96):
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size  # 이미지의 실제 크기 가져오기
    for box, scores in zip(pred_boxes, pred_scores):
        max_score = np.max(scores)  # 클래스별 점수 중 최대값
        if max_score > threshold:
            category_id = np.argmax(scores[:-1])  # 배경 클래스 제외
            
            # 모델이 반환하는 좌표를 절대 좌표로 변환 (상대 좌표일 경우)
            x_min, y_min, box_width, box_height = box
            x_min *= image_width
            y_min *= image_height
            box_width *= image_width
            box_height *= image_height

            x_max = x_min + box_width
            y_max = y_min + box_height

            draw.rectangle((x_min, y_min, x_max, y_max), outline=color, width=2)
            draw.text((x_min, y_min), id2label[category_id], fill=color)  # 예측 상자를 해당 색상으로

# IoU 계산 함수
def compute_iou(box1, box2):
    # box1, box2: (x_min, y_min, x_max, y_max) 형식의 경계 상자
    x_min1, y_min1, x_max1, y_max1 = box1
    x_min2, y_min2, x_max2, y_max2 = box2

    # 교차 영역의 좌상단, 우하단 좌표
    inter_x_min = max(x_min1, x_min2)
    inter_y_min = max(y_min1, y_min2)
    inter_x_max = min(x_max1, x_max2)
    inter_y_max = min(y_max1, y_max2)

    # 교차 영역의 너비와 높이
    inter_w = max(0, inter_x_max - inter_x_min)
    inter_h = max(0, inter_y_max - inter_y_min)

    # 교차 영역의 면적
    inter_area = inter_w * inter_h

    # 두 상자의 면적 계산
    box1_area = (x_max1 - x_min1) * (y_max1 - y_min1)
    box2_area = (x_max2 - x_min2) * (y_max2 - y_min2)

    # IoU 계산 (교차 영역 / (두 상자의 합에서 교차 영역을 뺀 값))
    union_area = box1_area + box2_area - inter_area
    iou = inter_area / union_area if union_area != 0 else 0

    return iou

# AP 계산 함수
def calculate_ap(pred_boxes, pred_scores, gt_boxes, iou_threshold=iouval):
    sorted_indices = np.argsort(-pred_scores)
    pred_boxes = pred_boxes[sorted_indices]

    tp = np.zeros(len(pred_boxes))
    fp = np.zeros(len(pred_boxes))
    matched = []

    for i, pred_box in enumerate(pred_boxes):
        max_iou = 0
        matched_gt_idx = -1
        for j, gt_box in enumerate(gt_boxes):
            if j not in matched:
                iou = compute_iou(pred_box, gt_box)
                if iou > max_iou:
                    max_iou = iou
                    matched_gt_idx = j
        
        if max_iou >= iou_threshold:
            tp[i] = 1
            matched.append(matched_gt_idx)
        else:
            fp[i] = 1

    # 누적 True Positive와 False Positive
    tp_cumsum = np.cumsum(tp)
    fp_cumsum = np.cumsum(fp)

    precision = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-6)
    recall = tp_cumsum / len(gt_boxes)

    ap = np.sum((recall[1:] - recall[:-1]) * precision[1:])  # AP 계산

    return ap

# 이미지와 라벨 파일 랜덤으로 5개 선택
image_files = os.listdir(image_folder)
random_files = random.sample(image_files, 5)  # 이미지 파일에서 랜덤으로 5개 선택

for image_file_name in random_files:
    image_file = os.path.join(image_folder, image_file_name)
    label_file_name = image_file_name.replace(".jpg", ".txt")
    label_file = os.path.join(label_folder, label_file_name)

    # 이미지 로드
    image = Image.open(image_file)
    
    # 이미지 전처리
    processed_image = custom_preprocess(image).to(device)

    # 기존 모델 예측
    with torch.no_grad():
        outputs_old = model_old(pixel_values=processed_image)

    # 새로운 모델 예측
    with torch.no_grad():
        outputs_new = model_new(pixel_values=processed_image)

    # 예측 경계 상자 및 점수
    pred_boxes_old = outputs_old["pred_boxes"][0].cpu().numpy()
    pred_scores_old = outputs_old["logits"].softmax(-1)[0].cpu().numpy()

    pred_boxes_new = outputs_new["pred_boxes"][0].cpu().numpy()
    pred_scores_new = outputs_new["logits"].softmax(-1)[0].cpu().numpy()

    # 정답 경계 상자 로드
    gt_boxes = []
    with open(label_file, "r") as f:
        labels = f.readlines()
        for label in labels:
            class_id, x_center, y_center, box_width, box_height = map(float, label.split())
            x_min = x_center - (box_width / 2)
            y_min = y_center - (box_height / 2)
            x_max = x_center + (box_width / 2)
            y_max = y_center + (box_height / 2)
            gt_boxes.append([x_min, y_min, x_max, y_max])

    # mAP 계산 (기존 모델)
    ap_old = calculate_ap(pred_boxes_old, pred_scores_old[:, :-1].max(axis=1), gt_boxes, iou_threshold=iouval)

    # mAP 계산 (새 모델)
    ap_new = calculate_ap(pred_boxes_new, pred_scores_new[:, :-1].max(axis=1), gt_boxes, iou_threshold=iouval)

    print(f"{image_file_name} - mAP (Old Model): {ap_old:.4f}, mAP (New Model): {ap_new:.4f}")

    # 정답 상자 그리기 (빨간색, YOLO 형식 라벨)
    draw_yolo_labels(image, label_file, id2label)
    Vth = 0.92
    # 기존 모델의 예측 상자 그리기 (파란색)
    draw_predictions(image, pred_boxes_old, pred_scores_old, id2label, color="blue", threshold=Vth)

    # 새로운 모델의 예측 상자 그리기 (초록색)
    draw_predictions(image, pred_boxes_new, pred_scores_new, id2label, color="green", threshold=Vth)

    # 최종 결과 출력
    image.show()


b53dac1763d49078be442e9e.jpg - mAP (Old Model): 99.6667, mAP (New Model): 99.6667
adc8bb32d8b0089a2f82eec1.jpg - mAP (Old Model): 37.3750, mAP (New Model): 37.3750
dda4f68c18e5e33eeaaeb37b.jpg - mAP (Old Model): 99.6667, mAP (New Model): 99.6667
da0c573d1a9f291c7d9951ec.jpg - mAP (Old Model): 299.0000, mAP (New Model): 299.0000
bab047c1dbc3abb60b3d5ca5.jpg - mAP (Old Model): 149.5000, mAP (New Model): 149.5000


In [19]:
import os
import random
import torch
from PIL import Image, ImageDraw
from transformers import AutoModelForObjectDetection
import numpy as np
import torchvision

# 경로 설정
image_folder = "C:/Users/fun67/Desktop/lastpj/data/train/images"
label_folder = "C:/Users/fun67/Desktop/lastpj/data/train/labels"

# 두 개의 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 기존 모델 로드
model_old = AutoModelForObjectDetection.from_pretrained(
    r"C:\Users\fun67\Desktop\lastpj\ckp\ckpt_rtdetr_r50vd_n98epoch"
).eval().to(device)

# 새로운 모델 로드
model_new = AutoModelForObjectDetection.from_pretrained(
    r"C:\Users\fun67\Desktop\lastpj\ckp\ckpt_rtdetr_r50vd_1epoch"
).eval().to(device)

# COCO 카테고리 id2label
id2label = {0: "crack", 1: "pothole"}

# 전처리 함수 정의
def custom_preprocess(image: Image.Image, target_size=(640, 640)):
    image = image.convert("RGB")
    image = image.resize(target_size)
    image = np.array(image).astype(np.float32) / 255.0
    image = np.transpose(image, (2, 0, 1))  # 채널을 첫 번째로 (C, H, W) 형식으로 변경
    return torch.tensor(image).unsqueeze(0)  # 배치 차원 추가

# YOLO 라벨 파일을 읽어서 경계 상자를 그리기
def draw_yolo_labels(image, label_file, id2label):
    image_width, image_height = image.size
    with open(label_file, "r") as f:
        labels = f.readlines()

    draw = ImageDraw.Draw(image)

    for label in labels:
        class_id, x_center, y_center, box_width, box_height = map(float, label.split())

        # YOLO 형식의 상대 좌표를 절대 좌표로 변환
        x_center *= image_width
        y_center *= image_height
        box_width *= image_width
        box_height *= image_height

        # 좌상단 (x_min, y_min) 및 우하단 (x_max, y_max) 계산
        x_min = x_center - (box_width / 2)
        y_min = y_center - (box_height / 2)
        x_max = x_center + (box_width / 2)
        y_max = y_center + (box_height / 2)

        # 빨간색으로 정답 상자 그리기
        draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
        draw.text((x_min, y_min), id2label[int(class_id)], fill="red")

# NMS 적용 함수
def apply_nms(pred_boxes, pred_scores, iou_threshold=0.5):
    """
    Apply NMS to the predicted boxes and return the filtered boxes and scores.
    pred_boxes: [num_boxes, 4] (x_min, y_min, x_max, y_max)
    pred_scores: [num_boxes] Scores for the boxes
    iou_threshold: IoU threshold for NMS
    """
    # pred_boxes는 x_min, y_min, x_max, y_max 형식이어야 함
    boxes = torch.tensor(pred_boxes)
    scores = torch.tensor(pred_scores)

    # Apply NMS
    keep = torchvision.ops.nms(boxes, scores, iou_threshold)

    # NMS 이후 선택된 박스와 점수만 반환
    nms_boxes = boxes[keep].numpy()
    nms_scores = scores[keep].numpy()

    return nms_boxes, nms_scores

# 모델 예측 상자 그리기 (각 모델마다 다른 색상)
# 모델 예측 상자 그리기 (각 모델마다 다른 색상)
def draw_predictions(image, pred_boxes, pred_scores, id2label, color="blue", threshold=0.96):
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size  # 이미지의 실제 크기 가져오기

    for box, score in zip(pred_boxes, pred_scores):
        if score > threshold:
            # 모델이 반환하는 좌표가 (x_min, y_min, x_max, y_max) 형식이어야 함
            x_min, y_min, x_max, y_max = box

            # 좌표가 유효한지 확인 (x_min < x_max, y_min < y_max)
            if x_min >= x_max or y_min >= y_max:
                print(f"Invalid box: {x_min, y_min, x_max, y_max}")
                continue  # 유효하지 않은 상자는 건너뜀

            # 좌표를 이미지 크기로 맞춤
            x_min *= image_width
            y_min *= image_height
            x_max *= image_width
            y_max *= image_height

            # 상자 그리기
            draw.rectangle([x_min, y_min, x_max, y_max], outline=color, width=2)



# 이미지와 라벨 파일 랜덤으로 5개 선택
image_files = os.listdir(image_folder)
random_files = random.sample(image_files, 5)

for image_file_name in random_files:
    image_file = os.path.join(image_folder, image_file_name)
    label_file_name = image_file_name.replace(".jpg", ".txt")  # 라벨 파일은 같은 이름의 .txt 파일
    label_file = os.path.join(label_folder, label_file_name)

    # 이미지 로드
    image = Image.open(image_file)

    # 이미지 전처리
    processed_image = custom_preprocess(image).to(device)

    # 기존 모델을 사용한 예측
    with torch.no_grad():
        outputs_old = model_old(pixel_values=processed_image)

    # 새로운 모델을 사용한 예측
    with torch.no_grad():
        outputs_new = model_new(pixel_values=processed_image)

    # 기존 모델의 예측 경계 상자 및 점수 추출
    pred_boxes_old = outputs_old["pred_boxes"][0].cpu().numpy()  # 예측된 경계 상자
    pred_scores_old = outputs_old["logits"].softmax(-1)[0].cpu().numpy()[:, :-1].max(axis=1)  # 클래스 점수

    # 새로운 모델의 예측 경계 상자 및 점수 추출
    pred_boxes_new = outputs_new["pred_boxes"][0].cpu().numpy()  # 예측된 경계 상자
    pred_scores_new = outputs_new["logits"].softmax(-1)[0].cpu().numpy()[:, :-1].max(axis=1)  # 클래스 점수

    # NMS 적용
    pred_boxes_old_nms, pred_scores_old_nms = apply_nms(pred_boxes_old, pred_scores_old, iou_threshold=0.9)
    pred_boxes_new_nms, pred_scores_new_nms = apply_nms(pred_boxes_new, pred_scores_new, iou_threshold=0.9)

    # 정답 상자 그리기 (빨간색, YOLO 형식 라벨)
    draw_yolo_labels(image, label_file, id2label)

    Vth = 0.92  # threshold 값
    # 기존 모델의 예측 상자 그리기 (파란색)
    draw_predictions(image, pred_boxes_old_nms, pred_scores_old_nms, id2label, color="blue", threshold=Vth)

    # 새로운 모델의 예측 상자 그리기 (초록색)
    draw_predictions(image, pred_boxes_new_nms, pred_scores_new_nms, id2label, color="green", threshold=Vth)

    # 최종 결과 출력
    image.show()


Invalid box: (0.082807064, 0.37776, 0.12791897, 0.22509557)
Invalid box: (0.049391948, 0.15486304, 0.101005435, 0.09452658)
Invalid box: (0.45633823, 0.3862209, 0.31349543, 0.54638654)
Invalid box: (0.080100544, 0.8028721, 0.15504864, 0.19185144)
Invalid box: (0.22195293, 0.5200341, 0.4439971, 0.044366367)
Invalid box: (0.12922998, 0.8493648, 0.056174714, 0.09817126)
Invalid box: (0.052536696, 0.7536015, 0.10236366, 0.0965512)
Invalid box: (0.79590523, 0.38962007, 0.072790995, 0.009856649)
Invalid box: (0.20135707, 0.13629088, 0.18739448, 0.0074338177)
Invalid box: (0.11482073, 0.5329275, 0.22793204, 0.020217948)
Invalid box: (0.17671977, 0.55657476, 0.014161417, 0.04917904)
Invalid box: (0.09442226, 0.80976, 0.12214617, 0.17619342)
Invalid box: (0.72727525, 0.34020004, 0.05858771, 0.0074990666)
Invalid box: (0.20150556, 0.7418025, 0.13736217, 0.08059214)
Invalid box: (0.7293525, 0.3408386, 0.054283194, 0.0071410476)
Invalid box: (0.14383672, 0.7789399, 0.023999646, 0.0075286455)
Inval